<a href="https://colab.research.google.com/github/fcggamou/ML_Crypto/blob/main/colab/ICD10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install icd10-cm
!pip install sentence-transformers

In [2]:
!git clone https://github.com/fcggamou/ICD10.git

Cloning into 'ICD10'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
import icd10
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample, models, util, models, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

In [ ]:
codes = [(key, value[1]) for key, value in icd10.codes.items()]
df = pd.DataFrame(codes, columns=['Code','Desc'])
df.set_index('Code', inplace=True, drop=True)
df

,Desc
Code,
A00,Cholera
A000,"Cholera due to Vibrio cholerae 01, biovar chol..."
A001,"Cholera due to Vibrio cholerae 01, biovar eltor"
A009,"Cholera, unspecified"
A01,Typhoid and paratyphoid fevers
...,...
Z91A28,Caregiver's intentional underdosing of medicat...
Z91A3,Caregiver's unintentional underdosing of patie...
Z91A4,Caregiver's other noncompliance with patient's...


In [ ]:
from torch import nn
word_embedding_model = models.Transformer('Emran/ClinicalBERT_description_full_ICD10_Code' , max_seq_length=64)
#word_embedding_model = models.Transformer('m' , max_seq_length=64)

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

# train_dataloader = DataLoader(train, shuffle=True, batch_size=64)
# train_loss = losses.CosineSimilarityLoss(model)
# evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test, name='sts-dev')

# #Tune the model
# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=100, evaluation_steps=7000, evaluator=evaluator, save_best_model=True, output_path=dest_path)
# model.save('model')

Downloading:   0%|          | 0.00/712k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480M [00:00<?, ?B/s]

Some weights of the model checkpoint at Emran/ClinicalBERT_description_full_ICD10_Code were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/382 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
emb_prediction = model.encode('', convert_to_tensor=True, normalize_embeddings=True).cpu()
scores = util.dot_score(emb_prediction, terms_emb)
n = int(np.argmax(scores))

print(scores[0][n])
print(df.loc[term_codes[n]])

tensor(0.4528)
Desc    Superficial frostbite of nose
Name: T3302, dtype: object


In [ ]:
terms_emb = model.encode(df['Desc'], convert_to_tensor=True, normalize_embeddings=True).cpu()
term_codes = df.index

  

In [ ]:
a = icd10.find("H53.1")
a.chapter

'VII'

In [ ]:
!pip install snomed-ct-simple

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement snomed-ct-simple (from versions: none)
ERROR: No matching distribution found for snomed-ct-simple
